In [1]:
# Install the necessary packages using pip
!pip install scikit-optimize
!pip install imbalanced-learn
!pip install imblearn
!pip install seaborn
!pip install xgboost
!pip install joblib

from os import pipe
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from skopt import BayesSearchCV  # Import BayesianSearchCV
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from joblib import Memory
from sklearn.metrics import make_scorer, balanced_accuracy_score, f1_score
from imblearn.over_sampling import SMOTE

# Instantiate the SMOTE object
smote = SMOTE(random_state=42)

# Check if 'fit' and 'transform' methods exist
print("Methods available in SMOTE object:")
print(dir(smote))

# Check if fit and transform exist in the SMOTE object
print("Does 'fit' method exist?", hasattr(smote, 'fit'))
print("Does 'transform' method exist?", hasattr(smote, 'transform'))
!pip show imbalanced-learn



Methods available in SMOTE object:
['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__sklearn_tags__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_build_request_for_signature', '_check_X_y', '_check_feature_names', '_check_n_features', '_doc_link_module', '_doc_link_template', '_doc_link_url_param_generator', '_estimator_type', '_fit_resample', '_generate_samples', '_get_default_requests', '_get_doc_link', '_get_metadata_request', '_get_param_names', '_get_tags', '_in_danger_noise', '_make_samples', '_more_tags', '_parameter_constraints', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_sampling_strategy_docstring', '_sa

In [2]:
from sklearn import set_config

set_config(enable_metadata_routing=False)
# Import necessary libraries
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Create a synthetic imbalanced dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a pipeline with SMOTE and a RandomForestClassifier
pipeline = Pipeline([
    ('smote', SMOTE(sampling_strategy='auto', random_state=42)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the classifier
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.94      0.96       275
           1       0.53      0.76      0.62        25

    accuracy                           0.92       300
   macro avg       0.75      0.85      0.79       300
weighted avg       0.94      0.92      0.93       300



In [ ]:


# Define cache directory
cache_dir = './pipeline_cache'
memory = Memory(location=cache_dir, verbose=0)
from sklearn import set_config

set_config(enable_metadata_routing=False)
# prompt: make another where target variable TARGET is excluded

df = pd.read_csv('application_train.csv')
print("Data Shape:", df.shape)
print("\nData Info:")
print(df.info())

X = df.drop('TARGET', axis=1)
y = df['TARGET']

from skopt.space import Real, Integer, Categorical

param_distributions = {
    'Logistic Regression': {
        'C': Real(1e-5, 100, prior='log-uniform'),
        'max_iter': Categorical([300]),
       # 'n_jobs': Categorical([-1]),
        'solver': Categorical(['lbfgs', 'saga', 'liblinear', 'newton-cg']),
    },
    'Random Forest': {
        'n_estimators': Integer(5, 150),
        'max_depth': Integer(5, 100),
        'min_samples_split': Integer(2, 20),
        'min_samples_leaf': Integer(1, 10),
        'bootstrap': Categorical([True, False]),
        'max_features': Categorical(['sqrt', 'log2', None]),
        'class_weight': Categorical([None, 'balanced']),
        'n_jobs': Categorical([-1]),
    },
    'K-Nearest Neighbors': {
        'n_neighbors': Integer(1, 200),
        'weights': Categorical(['uniform', 'distance']),
        'leaf_size': Integer(10, 100),
        'p': Integer(1, 5),
        'n_jobs': Categorical([-1]),
    },
    'XGBoost': {
        'subsample': Real(0.3, 1.0),
        'n_estimators': Integer(5, 150),
        'max_depth': Integer(3, 15),
        'learning_rate': Real(0.001, 0.5, prior='log-uniform'),
        'gamma': Real(0, 1.0),
        'colsample_bytree': Real(0.3, 1.0),
        'reg_alpha': Real(0, 10),
        'reg_lambda': Real(0, 10),
        'scale_pos_weight': Real(1, 10),
        'n_jobs': Categorical([-1]),
    }
}


models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'XGBoost': xgb.XGBClassifier(eval_metric='logloss')
}


# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(include=['object']).columns


# Preprocessing and transformers
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
]
                                 #, memory=memory
                                )  # Enable caching

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]
                                  # , memory=memory
                                  )  # Enable caching

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])



from sklearn.ensemble import RandomForestClassifier

# Create pipelines with caching enabled
def create_pipelines():
    pre_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
    ])

    smote_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
    ])

    rfe_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('feature_selection', RFE(estimator=LogisticRegression(max_iter=150, random_state=42, n_jobs=-1), n_features_to_select=10)),
    ], memory=memory)
    
    pca_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('dim_reduction', PCA(n_components=71, random_state=42)),
    ])

    rfe_smote_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('feature_selection', RFE(estimator=LogisticRegression(max_iter=150, random_state=42, n_jobs=-1), n_features_to_select=10)),
    ], memory=memory)

    pca_smote_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('dim_reduction', PCA(n_components=71, random_state=42)),
    ])


    return {
        "PCA": pca_pipeline,
        "PCA_SMOTE": pca_smote_pipeline,
        "RFE": rfe_pipeline, 
        "RFE_SMOTE": rfe_smote_pipeline,
        "Preprocessing": pre_pipeline, 
        "SMOTE": smote_pipeline
        }

import concurrent.futures
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.pipeline import Pipeline as ImbPipeline
from concurrent.futures import ProcessPoolExecutor


# Model evaluation function without parallelization
def evaluate_pipelines(models, pipelines, X_train, y_train, X_test, y_test):
    results = {}
    plt.figure(figsize=(12, 10))

    def evaluate_pipeline_model(pipeline_name, pipeline, model_name, model):
        print(f"[{pipeline_name} - {model_name}] Training started...")

        best_model = None
        pipeline_with_model = None
        if param_distributions and model_name in param_distributions:
            print(f"[{pipeline_name} - {model_name}] Performing Bayesian search...")
            bayes_search = BayesSearchCV(
                model,
                param_distributions[model_name],
                n_iter=60,
                n_jobs=20,
                n_points=12,
                cv=2,
                random_state=42,
                scoring=make_scorer(f1_score, pos_label=1),  # F1 score for the minority class (class 1)
                verbose=5
            )
            pipeline_with_model = Pipeline(steps=pipeline.steps + [('classifier', bayes_search)])
            pipeline_with_model.fit(X_train, y_train)
            best_model = bayes_search.best_estimator_
            print(f"[{pipeline_name} - {model_name}] Best hyperparameters: {bayes_search.best_params_}")
        else:
            print(f"[{pipeline_name} - {model_name}] No hyperparameter tuning.")
            pipeline_with_model = Pipeline(steps=pipeline.steps + [('classifier', model)])
            pipeline_with_model.fit(X_train, y_train)
            best_model = model
        

        #print(f"[{pipeline_name} - {model_name}] Using default parameters...")
        #pipeline_with_model = Pipeline(steps=pipeline.steps[:-1] + [('classifier', model)])
        #pipeline_with_model.fit(X_train, y_train)
        #best_model = model


        print(f"[{pipeline_name} - {model_name}] Evaluating model...")
        pipeline = pipeline_with_model
        # Predictions and metrics
        y_pred_prob = pipeline.predict_proba(X_test)[:, 1]
        y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_prob)
        print(f"[{pipeline_name} - {model_name}] Accuracy: {accuracy:.4f}, ROC AUC: {roc_auc:.4f}")
        print(f"[{pipeline_name} - {model_name}] Classification Report:\n{classification_report(y_test, y_pred)}")
        print(f"[{pipeline_name} - {model_name}] Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")

        # Store results
        key = f"{model_name}_{pipeline_name}"
        results[key] = {
            "Accuracy": accuracy,
            "ROC AUC": roc_auc,
            "Classification Report": classification_report(y_test, y_pred)
        }

        # Plot ROC curve
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        plt.plot(fpr, tpr, label=f'{key} (AUC = {roc_auc:.2f})')

    # Sequential execution of each pipeline-model combination
    for pipeline_name, pipeline in pipelines.items():
        for model_name, model in models.items():
            evaluate_pipeline_model(pipeline_name, pipeline, model_name, model)

    # Plot settings
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Models and Pipelines')
    plt.legend(loc='lower right')
    plt.show()

    return results



# Create pipelines
pipelines = create_pipelines()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Evaluate models with pipelines
results = evaluate_pipelines(models, pipelines, X_train, y_train, X_test, y_test)

# Print results
for key, metrics in results.items():
    print(f"\n{key} Performance:")
    print(f"Accuracy: {metrics['Accuracy']:.4f}")
    print(f"ROC AUC: {metrics['ROC AUC']:.4f}")
#    print(f"CV Mean ROC AUC: {metrics['CV Mean ROC AUC']:.4f} (+/- {metrics['CV Std ROC AUC']:.4f})")
    print(f"Classification Report:\n{metrics['Classification Report']}")


Data Shape: (307511, 122)

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB
None
[PCA - Logistic Regression] Training started...
[PCA - Logistic Regression] Performing Bayesian search...
Fitting 2 folds for each of 12 candidates, totalling 24 fits


              precision    recall  f1-score   support

           0       0.98      0.94      0.96       275
           1       0.53      0.76      0.62        25

    accuracy                           0.92       300
   macro avg       0.75      0.85      0.79       300
weighted avg       0.94      0.92      0.93       300



In [ ]:
Data Shape: (307511, 122)

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB
None
[PCA - Logistic Regression] Training started...
[PCA - Logistic Regression] Performing Bayesian search...
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Fitting 2 folds for each of 12 candidates, totalling 24 fits
[PCA - Logistic Regression] Best hyperparameters: OrderedDict([('C', 99.52032142391181), ('max_iter', 300), ('solver', 'lbfgs')])
[PCA - Logistic Regression] Evaluating model...
[PCA - Logistic Regression] Accuracy: 0.9193, ROC AUC: 0.7419
[PCA - Logistic Regression] Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.51      0.01      0.02      4965

    accuracy                           0.92     61503
   macro avg       0.72      0.50      0.49     61503
weighted avg       0.89      0.92      0.88     61503

[PCA - Logistic Regression] Confusion Matrix:
[[56501    37]
 [ 4926    39]]
[PCA - Random Forest] Training started...
[PCA - Random Forest] Performing Bayesian search...
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Fitting 2 folds for each of 12 candidates, totalling 24 fits
[PCA - Random Forest] Best hyperparameters: OrderedDict([('bootstrap', True), ('class_weight', 'balanced'), ('max_depth', 7), ('max_features', 'log2'), ('min_samples_leaf', 6), ('min_samples_split', 20), ('n_estimators', 83), ('n_jobs', -1)])
[PCA - Random Forest] Evaluating model...
[PCA - Random Forest] Accuracy: 0.7004, ROC AUC: 0.7130
[PCA - Random Forest] Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.71      0.81     56538
           1       0.15      0.61      0.25      4965

    accuracy                           0.70     61503
   macro avg       0.55      0.66      0.53     61503
weighted avg       0.89      0.70      0.77     61503

[PCA - Random Forest] Confusion Matrix:
[[40074 16464]
 [ 1960  3005]]
[PCA - K-Nearest Neighbors] Training started...
[PCA - K-Nearest Neighbors] Performing Bayesian search...
Fitting 2 folds for each of 12 candidates, totalling 24 fits